In [ ]:
import sqlite3

conn = sqlite3.connect("../data/processed/database")

In [ ]:
cursor = conn.cursor()

In [ ]:
import requests 

from io import StringIO

import pandas as pd

request = requests.get("https://raw.githubusercontent.com/leogermani/estados-e-municipios-ibge/master/estados.csv")

data = StringIO(request.text)

df = pd.read_csv(data, sep=",")

df

,COD,NOME,SIGLA
0,35,São Paulo,SP
1,41,Paraná,PR
2,42,Santa Catarina,SC
3,43,Rio Grande do Sul,RS
4,50,Mato Grosso do Sul,MS
5,11,Rondônia,RO
6,12,Acre,AC
7,13,Amazonas,AM
8,14,Roraima,RR
9,15,Pará,PA


In [ ]:
cursor.execute(
    """ 
    CREATE TABLE estados_nao_processado (
      sigla CHAR[3] PRIMARY KEY,
      nome TEXT
    ); 
    """
)

In [ ]:
from tqdm import tqdm
def dataframe_to_sql(
  df,
  table_name,
  conn,
  cursor,
  map_columns
):

  TABLE_VALUES = ",".join(list(map_columns.values()))

  INSERT_STRING = f"INSERT INTO {table_name}({{}}) VALUES ({{}})"


  for _, i in tqdm(df.loc[:, map_columns.keys()].iterrows()):
    
    TABLE_VALUES = ",".join(
        value for key, value in map_columns.items() if not pd.isnull(i[key])
    )

    VALUES = ",".join(
      [f'"{value}"' if (type(value) is str) else str(value) for value in i.values if not pd.isnull(value)]
    )

    cursor.execute(INSERT_STRING.format(TABLE_VALUES, VALUES))

  conn.commit()


In [ ]:
dataframe_to_sql(
    df=df,
    table_name="estados_nao_processado",
    conn=conn,
    cursor=cursor,
    map_columns={
        "NOME": "nome",
        "SIGLA": "sigla"
    }
)

In [ ]:
cursor.execute(
    "CREATE TABLE estados AS "\
    "SELECT substr(sigla, 2, 3) AS sigla, nome from estados_nao_processado"
)

In [ ]:
cursor.execute("""
  CREATE TABLE fronteiras (
    id INTEGER PRIMARY KEY,
    sigla_1 CHAR[2],
    sigla_2 CHAR[2]
  )
""")

In [ ]:
df = pd.read_csv("../data/external/fronteiras.csv")

dataframe_to_sql(
    df=df,
    table_name="fronteiras",
    conn=conn,
    cursor=cursor,
    map_columns={
        "sigla1": "sigla_1",
        "sigla2": "sigla_2"
    }
)

In [ ]:
cursor.execute("""CREATE TABLE dados_nao_processados (
    estado TEXT,
    data TEXT,
    semanaEpi INTEGER,
    populacaoTCU2019 REAL,
    casosAcumulado INTEGER,
    casosNovos INTEGER,
    obitosAcumulado INTEGER,
    obitosNovos INTEGER
);""")

In [ ]:
file_list = [
  "../data/external/HIST_PAINEL_COVIDBR_2020_Parte1_10nov2021.csv",
  "../data/external/HIST_PAINEL_COVIDBR_2020_Parte2_10nov2021.csv",
  "../data/external/HIST_PAINEL_COVIDBR_2021_Parte1_10nov2021.csv",
  "../data/external/HIST_PAINEL_COVIDBR_2021_Parte2_10nov2021.csv",
]

for file_ in file_list:
  df = pd.read_csv(file_, sep=";")

  dataframe_to_sql(
      df=df,
      conn=conn,
      cursor=cursor,
      table_name="dados_nao_processados",
      map_columns={
          "estado": "estado",
          "data": "data",
          "semanaEpi": "semanaEpi",
          "populacaoTCU2019": "populacaoTCU2019",
          "casosAcumulado": "casosAcumulado",
          "casosNovos": "casosNovos",
          "obitosAcumulado": "obitosAcumulado",
          "obitosNovos": "obitosNovos"
      }
  )



714481it [02:04, 5752.56it/s]
859707it [02:29, 5750.96it/s]
1017039it [02:58, 5702.68it/s]
747327it [02:09, 5762.95it/s]


In [ ]:
cursor.execute(
    "CREATE TABLE dados AS "\
    "SELECT * FROM dados_nao_processados "\
    "WHERE estado IS NOT NULL"
)

In [ ]:
cursor.execute( 
    "CREATE TABLE dados_estados_total AS "\
    "SELECT estado, MAX(casosAcumulado) AS casosAcumulados, MAX(obitosAcumulado) as obitosAcumulados "\
    "FROM dados GROUP BY estado"
)

In [ ]:
cursor.execute(
    """
    CREATE TABLE pessoas_habilitadas_nao_processado (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      data VARCHAR(8),
      contagem_pessoas INTEGER
    );
    """
)

In [ ]:
df = pd.read_csv("../data/external/pessoas_habilitadas.csv")

dataframe_to_sql(
    df=df,
    conn=conn,
    cursor=cursor,
    table_name="pessoas_habilitadas_nao_processado",
    map_columns={
        'Data': 'data',
         'Pessoas de 14 anos ou mais de idade - Pessoa - Instituto Brasileiro de Geografia e Estatística': 'contagem_pessoas'
    }
)

114it [00:00, 6959.27it/s]


In [ ]:
cursor.execute(
    "CREATE TABLE pessoas_habilitadas AS "\
    "SELECT * from pessoas_habilitadas_nao_processado WHERE data > \"2019\";"
)

In [ ]:
cursor.execute(
    """
    CREATE TABLE pessoas_ocupadas_nao_processado (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      data VARCHAR(8),
      contagem_pessoas INTEGER
    );
    """
)

In [ ]:
df = pd.read_csv("../data/external/pessoas_ocupadas.csv")

dataframe_to_sql(
    df=df,
    conn=conn,
    cursor=cursor,
    table_name="pessoas_ocupadas_nao_processado",
    map_columns={
        'Data': 'data',
        'Pessoas ocupadas - Pessoa - Instituto Brasileiro de Geografia e Estatística': 'contagem_pessoas'
    }
)

114it [00:00, 5978.75it/s]


In [ ]:
cursor.execute(
    "CREATE TABLE pessoas_ocupadas AS "\
    "SELECT * from pessoas_ocupadas_nao_processado WHERE data > \"2019\";"
)

In [ ]:
cursor.execute(
    "CREATE TABLE data_economico AS "\
    "SELECT pessoas_ocupadas.data as data, "\
    "pessoas_ocupadas.contagem_pessoas as contagem_pessoas_ocupadas, "\
    "pessoas_habilitadas.contagem_pessoas as contagem_pessoas_habilitadas "\
    "FROM pessoas_ocupadas INNER JOIN pessoas_habilitadas ON pessoas_ocupadas.data = pessoas_habilitadas.data"
)